In [125]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
import validation
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss
from sklearn.pipeline import make_pipeline



In [133]:
to_predict_mens = pd.read_csv("to_predict_mens.csv")

In [134]:
to_predict_mens[(to_predict_mens["GameRound"] == 1) & (to_predict_mens["Season"] == 2024)][["t1_TeamName", "t2_TeamName", "final_odds"]].head()

,t1_TeamName,t2_TeamName,final_odds
2644,Connecticut,Stetson,-26.5
2705,Iowa St,S Dakota St,-16.5
2765,Illinois,Morehead St,-12.0
2824,Auburn,Yale,-13.0
2882,San Diego St,UAB,-7.0


In [173]:
pd.set_option('display.max_columns', None)
1345
to_predict_mens[(to_predict_mens["Season"] == 2024)][["Team1", "t1_TeamName", "t1_adj_margin"]].drop_duplicates().sort_values(by="t1_adj_margin")

,Team1,t1_TeamName,t1_adj_margin
2921,1246,Kentucky,-407.056815
2647,1124,Baylor,-367.850338
2879,1321,Northwestern,-359.884023
2903,1345,Purdue,-349.947535
3024,1400,Texas,-346.895073
...,...,...,...
2722,1224,Howard,243.775995
2934,1255,Longwood,249.178582
3032,1389,St Peter's,274.067461
2884,1287,Morehead St,299.484637


In [169]:
to_predict_mens[(to_predict_mens["GameRound"] == 1) & (to_predict_mens["Season"] == 2024)
                & (to_predict_mens["Team2"] == 1345)]

,Unnamed: 0,type,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_TeamName,t1_FirstD1Season,t1_LastD1Season,t2_TeamName,t2_FirstD1Season,t2_LastD1Season,final_odds,GameRound,t1_FGM,t1_FGA,t1_FGM3,t1_FGA3,t1_OR,t1_Ast,t1_TO,t1_Stl,t1_PF,t1_FTA,t1_FTM,t1_PointDiff,t2_FGM,t2_FGA,t2_FGM3,t2_FGA3,t2_OR,t2_Ast,t2_TO,t2_Stl,t2_PF,t2_FTA,t2_FTM,t2_PointDiff,t1_OrdinalRank,t2_OrdinalRank,t1_Seed,t2_Seed,seed_diff,t1_adj_oe,t1_adj_de,t1_adj_margin,t2_adj_oe,t2_adj_de,t2_adj_margin,t1_final_rank,t2_final_rank
4164,4164,Prediction,2024_1212_1345,0.5,2024,1212,1345,NaN,M,NaN,Grambling,1985,2024,Purdue,1985,2024,24.5,1,22.580645,52.354839,5.322581,15.612903,7.741935,9.16129,12.741935,7.129032,16.16129,20.741935,14.645161,-3.741935,28.515152,58.393939,8.333333,20.424242,11.030303,18.393939,10.969697,5.666667,14.363636,25.0,18.030303,13.242424,25.0,2.0,16,1,15,209.522761,-176.938036,386.460797,-117.666782,232.280753,-349.947535,65.379039,97.181957


### Prepare data 

In [135]:
def prepare_data(to_predict_mens, season):

    to_predict_mens_first_round_train = to_predict_mens[(to_predict_mens["GameRound"] == 1)
                                                        & (to_predict_mens.final_odds.notnull())
                                                        & (to_predict_mens.Season < season)
                                                        ].copy()
    
    to_predict_mens_train = to_predict_mens[(to_predict_mens.Season < season)].copy()

    to_predict_mens_first_round_test = to_predict_mens[(to_predict_mens.Season == season)
                                                    & (to_predict_mens.GameRound == 1)
                                                    & (to_predict_mens.final_odds.notnull())
                                                    ].copy()

    to_predict_mens_other_rounds_test = to_predict_mens[(to_predict_mens.Season == season)
                                                    & (to_predict_mens.GameRound > 1)
                                                    ].copy()
    
    return to_predict_mens_first_round_train, to_predict_mens_train, to_predict_mens_first_round_test, to_predict_mens_other_rounds_test
    


### Odds Model Training

In [136]:
def train_odds_model(to_predict_mens_first_round_train):

    best_params = {"C": .1}
    model = LogisticRegression(**best_params)
    pipeline = make_pipeline(StandardScaler(), model)
    odds_model = pipeline.fit(to_predict_mens_first_round_train[["final_odds"]], to_predict_mens_first_round_train["Outcome"])
    
    return odds_model


### Statistics Model Training

In [137]:
def train_statistics_model(to_predict_mens_train, statistics_features):

    best_params = {"C": .1}
    model = LogisticRegression(**best_params)
    pipeline = make_pipeline(StandardScaler(), model)
    statistics_model = pipeline.fit(to_predict_mens_train[statistics_features], to_predict_mens_train["Outcome"])

    return statistics_model


### Inference

In [138]:
def inference(to_predict_mens_first_round_test, to_predict_mens_other_rounds_test, odds_model, statistics_model, features):

    pred_proba = odds_model.predict_proba(to_predict_mens_first_round_test[["final_odds"]].copy())[:,1]
    to_predict_mens_first_round_test["odds_pred"] = pred_proba

    pred_proba = statistics_model.predict_proba(to_predict_mens_first_round_test[features].copy())[:,1]
    to_predict_mens_first_round_test["statistics_pred"] = pred_proba

    to_predict_mens_first_round_test["Pred"] = (to_predict_mens_first_round_test.odds_pred * .75) + \
                                                (to_predict_mens_first_round_test.statistics_pred * .25)
    

   

    pred_proba = statistics_model.predict_proba(to_predict_mens_other_rounds_test[features].copy())[:,1]
    to_predict_mens_other_rounds_test["Pred"] = pred_proba

    mens_sub = pd.concat([to_predict_mens_first_round_test[["ID", "Pred"]],
            to_predict_mens_other_rounds_test[["ID", "Pred"]]], axis=0)
    
    return mens_sub



### Full Pipeline

In [139]:
def run(to_predict_mens, season, statistics_features):
    
    # get data
    to_predict_mens_first_round_train, to_predict_mens_train, to_predict_mens_first_round_test, to_predict_mens_other_rounds_test= prepare_data(to_predict_mens, season)
    
    # train
    odds_model = train_odds_model(to_predict_mens_first_round_train)
    statistics_model = train_statistics_model(to_predict_mens_train, statistics_features)

    # inference
    sub = inference(to_predict_mens_first_round_test, to_predict_mens_other_rounds_test, odds_model, statistics_model, statistics_features) 

    return sub
    

In [140]:
statistics_features = ['t1_adj_margin', 't2_adj_margin', 't1_final_rank', 't2_final_rank', 't1_OrdinalRank', 't2_OrdinalRank']
mens_sub = run(to_predict_mens, 2024, statistics_features)

### Womens

In [155]:
to_predict_women = pd.read_csv("to_predict_women.csv")

to_predict_women_train = to_predict_women[to_predict_women.Season != 2024] 
to_predict_women_test = to_predict_women[to_predict_women.Season == 2024] 


In [156]:
statistics_features = ['seed_diff', 't1_adj_margin', 't2_adj_margin']
best_params = {"C": .1}
model = LogisticRegression(**best_params)
pipeline = make_pipeline(StandardScaler(), model)
statistics_model = pipeline.fit(to_predict_women_train[statistics_features], to_predict_women_train["Outcome"])


In [157]:
pred_proba = statistics_model.predict_proba(to_predict_women_test[statistics_features].copy())[:,1]
to_predict_women_test["Pred"] = pred_proba
womens_sub = to_predict_women_test[["ID", "Pred"]]


/var/folders/2g/465yxy_x4g786jx2llr5xqh40000gn/T/ipykernel_13428/593629811.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict_women_test["Pred"] = pred_proba


In [159]:
final_sub = pd.concat([mens_sub, womens_sub], axis=0)

In [161]:
final_sub.to_csv("traditional_format_sub.csv")

In [88]:
# statistics_features = ['t1_adj_margin', 't2_adj_margin', 't1_final_rank', 't2_final_rank', 't1_OrdinalRank', 't2_OrdinalRank']

# all_subs = []

# for year in range(2010, 2024):
#     if year != 2020:
#         sub = run(to_predict_mens, year, statistics_features)
#         all_subs.append(sub)

#all_subs = pd.concat(all_subs)
#all_subs.to_csv("../submissions/all_sky_model_preds.csv")


In [61]:

#mens_sub.to_csv("../submissions/sub_2023_in_2024.csv")